# Importação de Bibliotecas

In [67]:

import pandas as pd
from pre_processing2 import Pipeline
from sklearn.datasets import fetch_20newsgroups

import re
import spacy

import nltk
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer


import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import Sparse2Corpus



In [2]:
### !scp /mnt/nas/trf1-dados

# Carregando os Dados

In [3]:

# newsgroup = fetch_20newsgroups(subset="train",remove=('headers', 'footers', 'quotes'))
# dados = pd.DataFrame(newsgroup.data)
# dados
dados = pd.read_parquet('/mnt/home/breno.beleza/datasets/stj-parquet.gzip')
dados = pd.DataFrame(dados['decisao'][:2000])
dados

decisao
0     EMENTA\nHABEAS CORPUS. CONDIÇÕES DO SURSIS. PR...
1     EMENTA\nRECURSO ORDINÁRIO CONSTITUCIONAL EM HA...
2     ACÓRDÃO\nVistos e relatados estes autos em que...
3     ACÓRDÃO\nVistos e relatados estes autos em que...
4     ACÓRDÃO\nVistos e relatados estes autos em que...
...                                                 ...
1995  DECISÃO<br>Trata-se de recurso especial interp...
1996  DECISÃO<br>Trata-se de recurso especial interp...
1997  DECISÃO<br>Cuida-sede embargos de declaração o...
1998  DECISÃO<br>Trata-se de agravo da FAZENDA NACIO...
1999  DECISÃO<br>ROBERTO DE FIGUEIREDO CALDAS agrava...

[2000 rows x 1 columns]

In [4]:
# dados = dados.reset_index(drop=True)

# Aplicando o pré processamento

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /mnt/home/breno.beleza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/home/breno.beleza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
stop_words = stopwords.words('portuguese')
# stop_words = stopwords.words('english')


In [8]:
first =['nacional','caso','lei','processo','direito','processual','civil','administrativo','tributário','agravo','instrumento','constitucional','penal','art','sentença','lei','mantida','nº']


stop_words = stop_words + first

def remove_stop_words(data_words,stop_words=stop_words):
    return [word for word in data_words if word not in stop_words]

In [49]:
def pre_process(dataframe,texto):
    dataframe['processed_text'] = dataframe[texto].apply(lambda x: Pipeline(disabled_steps=[ "transform_ner", "lemmatize" , "spellcheck", "stem", "transform_unknown", 'transform_syn'], embedding=False).apply(x, verbose=False))
    # dataframe['processed_text'] = dataframe[texto].apply(lambda x: Pipeline().apply(x, verbose=False))
    dataframe['processed_text'] = dataframe['processed_text'].apply(remove_stop_words)
    # dataframe = bigram_and_trigram(dataframe)    
    return dataframe.processed_text

In [50]:
dados_preprocessados = pre_process(dados,'decisao') ## Deve demorar alguns minutos

/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1001: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn("Unknown precedent: '%s'", precedent)
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1001: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn("Unknown precedent: '%s'", precedent)
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1484: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1484: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1001: DeprecationWarning: The 'warn

In [57]:
dados_preprocessados = pd.DataFrame(dados_preprocessados)
dados_preprocessados

processed_text
0     [ementa, habeas, corpus, condicoes, sursis, pr...
1     [ementa, recurso, ordinario, habeas, corpus, h...
2     [acordao, vistos, relatados, autos, partes, ac...
3     [acordao, vistos, relatados, autos, partes, ac...
4     [acordao, vistos, relatados, autos, partes, ac...
...                                                 ...
1995  [decisaobrtratase, recurso, especial, interpos...
1996  [decisaobrtratase, recurso, especial, interpos...
1997  [decisaobrcuidasede, embargos, declaracao, opo...
1998  [decisaobrtratase, fazenda, objetivaadmissao, ...
1999  [decisaobrroberto, figueiredo, caldas, agrava,...

[2000 rows x 1 columns]

# Aplicando o TF-IDF

In [58]:
tf_vectorizer = TfidfVectorizer(strip_accents='unicode', 
                                token_pattern= r'\b[a-zA-Z]{3,}\b',
                                max_df=0.5)

In [62]:
data_vectorized = tf_vectorizer.fit_transform(dados_preprocessados['processed_text'].apply(lambda x: ' '.join(x)).tolist())
# data_vectorized = dados_preprocessados.apply()
data_vectorized

<2000x50666 sparse matrix of type '<class 'numpy.float64'>'
	with 669359 stored elements in Compressed Sparse Row format>

## Modelagem de tópicos

In [78]:
def generate_lda(data_ready, number_of_topics, dados_preprocessados):
    
    # data_ready = pre_process(dataframe)
    id2word = corpora.Dictionary(dados_preprocessados['processed_text'])
    temp = id2word[0]
    corpus = [id2word.doc2bow(text) for text in dados_preprocessados['processed_text']]
    # corpus = gensim.matutils.Sparse2Corpus(data_ready, documents_columns=False)
    lda_model = LdaModel(corpus=corpus,
                         id2word=id2word,
                         num_topics=number_of_topics,
                         random_state=42,
                         update_every=1,
                         chunksize=len(corpus),
                         passes=10,
                         alpha='auto',
                         iterations=100)
    return lda_model, corpus, id2word


In [79]:
number_of_topics = 7
lda_model, corpus, id2word = generate_lda(data_vectorized, 7, dados_preprocessados)

In [80]:
viz_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.display(viz_data)


/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [ ]:
# pyLDAvis.sklearn.prepare(lda3, tfidf_vectorizer.fit_transform(dados_preprocessados_completo), tfidf_vectorizer)